In [ ]:
!nvidia-smi

Sun Nov 21 12:12:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip -q install torchaudio
# !pip -q install datasets

import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook as tqdm
from shutil import copyfile

import librosa
import librosa.display
import scipy.io.wavfile as wav

import IPython.display as ipd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
!pip3 install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.4 MB/s eta 0:11:22tcmalloc: large alloc 1147494400 bytes == 0x55bfa1b4e000 @  0x7f915a874615 0x55bf6869e4cc 0x55bf6877e47a 0x55bf686a12ed 0x55bf68792e1d 0x55bf68714e99 0x55bf6870f9ee 0x55bf686a2bda 0x55bf68714d00 0x55bf6870f9ee 0x55bf686a2bda 0x55bf68711737 0x55bf68793c66 0x55bf68710daf 0x55bf68793c66 0x55bf68710daf 0x55bf68793c66 0x55bf68710daf 0x55bf686a3039 0x55bf686e6409 0x55bf686a1c52 0x55bf68714c25 0x55bf6870f9ee 0x55bf686a2bda 0x55bf68711737 0x55bf6870f9ee 0x55bf686a2bda 0x55bf68710915 0x55bf686a2afa 0x55bf68710c0d 0x55bf6870f9ee
     |██████████████████▌             | 1055.7 MB 1.8 MB/s eta 0:06:58tcmalloc: large alloc 1434370048 bytes == 0x55bfe61a4000 @  0x7f915a874615 0x55bf6869e4cc 0x55bf6877e47a 0x55bf686a12ed 0x55bf68792e1d 0x55bf68714e99 0x55bf6870f9ee 0x55bf686a2bda 0x55bf68714d00 0x55bf6870f9ee 0x55bf686a2bda 0x55bf68711737 0x55bf68793c66 0x55bf

In [ ]:
import torch
torch.version.cuda


'11.3'

In [ ]:
import torchaudio

In [ ]:
import pandas as pd
from torch.utils.data import Dataset

import torch

In [ ]:
train_rc = pd.read_csv('/content/drive/MyDrive/Tesi/train_rc.csv')
test_rc = pd.read_csv('/content/drive/MyDrive/Tesi/test_rc.csv')

In [ ]:
!7z x /content/drive/MyDrive/Tesi/dataset_bandPass2.zip -o/content/


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Tesi/                                      1 file, 569072075 bytes (543 MiB)

Extracting archive: /content/drive/MyDrive/Tesi/dataset_bandPass2.zip
 59% 4096 Open              --
Path = /content/drive/MyDrive/Tesi/dataset_bandPass2.zip
Type = zip
Physical Size = 569072075

  0%      0% 48         1% 94 - dataset_bandPass2/110_1p1_Lr_sc_Meditron_2_bandPass2.wav                                                                    1% 133 - dataset_bandPass2/193_1b4_Lr_mc_AKGC417

In [ ]:
print(train_rc.shape)
print(test_rc.shape)

(5518, 2)
(1380, 2)


In [ ]:
print(train_rc['label'].unique())

['none' 'wheeze' 'crackle' 'both']


In [ ]:
train_4 = train_rc.copy()
train_4 = train_4.replace('none', 0)
train_4 = train_4.replace('crackle', 1)
train_4 = train_4.replace('wheeze', 2)
train_4 = train_4.replace('both',3)
train_4['filename']="/content/dataset_bandPass2/"+train_4['filename']+'_bandPass2.wav'
train_4

,filename,label
0,/content/dataset_bandPass2/194_1b1_Pr_sc_Medit...,0
1,/content/dataset_bandPass2/113_1b1_Ar_sc_Litt3...,0
2,/content/dataset_bandPass2/211_1p2_Pr_mc_AKGC4...,0
3,/content/dataset_bandPass2/120_1b1_Lr_sc_Medit...,0
4,/content/dataset_bandPass2/203_1p4_Pr_mc_AKGC4...,0
...,...,...
5513,/content/dataset_bandPass2/158_1p4_Tc_mc_AKGC4...,0
5514,/content/dataset_bandPass2/154_1b3_Ll_mc_AKGC4...,1
5515,/content/dataset_bandPass2/113_1b1_Lr_sc_Litt3...,0
5516,/content/dataset_bandPass2/204_2b5_Al_mc_AKGC4...,2


In [ ]:
test_4 = test_rc.copy()
test_4 = test_4.replace('none', 0)
test_4 = test_4.replace('crackle', 1)
test_4 = test_4.replace('wheeze', 2)
test_4 = test_4.replace('both',3)
test_4['filename']="/content/dataset_bandPass2/"+test_4['filename']+'_bandPass2.wav'
test_4

,filename,label
0,/content/dataset_bandPass2/157_1b1_Pr_sc_Medit...,1
1,/content/dataset_bandPass2/130_2b2_Al_mc_AKGC4...,1
2,/content/dataset_bandPass2/210_1b1_Ar_sc_Medit...,0
3,/content/dataset_bandPass2/163_8b3_Ll_mc_AKGC4...,0
4,/content/dataset_bandPass2/178_1b6_Pl_mc_AKGC4...,0
...,...,...
1375,/content/dataset_bandPass2/147_1b2_Tc_mc_AKGC4...,0
1376,/content/dataset_bandPass2/192_2b2_Al_mc_LittC...,3
1377,/content/dataset_bandPass2/130_1p2_Pl_mc_AKGC4...,1
1378,/content/dataset_bandPass2/200_2p3_Pr_mc_AKGC4...,0


In [ ]:
train_4.groupby('label').count()

,filename
label,
0,2946
1,1469
2,704
3,399


In [ ]:
test_4.groupby('label').count()

,filename
label,
0,696
1,395
2,182
3,107


In [ ]:
!pip -q install torchaudio

In [ ]:
import torchaudio
duration = 4
freq = 16
class AudiosDataset(Dataset):
    def __init__(self, paths=None, targets=None, augment=False) -> None:
        self.paths = paths
        self.targets = targets
        self.augment = augment
        
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> dict:
        y, sr = torchaudio.load(self.paths.iloc[idx])#, normalization=True)
        """
        if self.augment:
            effects = [
                    ["lowpass", "-1", "300"], 
                    ["speed", f"{random.uniform(0.7, 1.3)}"],  # change speed
                  ]
            y, sr = torchaudio.sox_effects.apply_effects_tensor(
                y, sr, effects)
        #
            
        if self.augment:
            # augment sound in order to imitate the room change
            rir = self.rir[:, int(16000*1.1):int(16000*1.3)]
            rir = rir / torch.norm(rir, p=2)
            rir = torch.flip(rir, [1])
            y = torch.nn.functional.conv1d(y[None, ...], rir[None, ...])[0]
        
        """
        # here more augmentations could be used
        y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=freq*1000)(y)

        """
        if self.augment:
            y = (0.5)*torch.randn(y.shape)
        """
        fixed_length = (freq*1000) * duration
        
        # returning result
        if y.shape[1] < fixed_length:
            y = torch.nn.functional.pad(
              y, (0, fixed_length - y.shape[1]))
        else:
            y = y[:, :fixed_length]
        result = {"target":self.targets.iloc[idx], "representation":y}
        return result

In [ ]:
train_ds = AudiosDataset(train_4['filename'], train_4['label'])
test_ds = AudiosDataset(test_4['filename'], test_4['label'])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    )
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
!pip install fairseq@git+https://github.com//pytorch/fairseq.git@f2146bdc7abf293186de9449bfa2272775e39e1d#egg=fairseq
!pip install sox

  Cloning https://github.com//pytorch/fairseq.git (to revision f2146bdc7abf293186de9449bfa2272775e39e1d) to /tmp/pip-install-wb3s6ux2/fairseq_7e585d10b021479ebc0e8ac2adbbaad9
  Running command git clone -q https://github.com//pytorch/fairseq.git /tmp/pip-install-wb3s6ux2/fairseq_7e585d10b021479ebc0e8ac2adbbaad9
  Running command git rev-parse -q --verify 'sha^f2146bdc7abf293186de9449bfa2272775e39e1d'
  Running command git fetch -q https://github.com//pytorch/fairseq.git f2146bdc7abf293186de9449bfa2272775e39e1d
  Running command git checkout -q f2146bdc7abf293186de9449bfa2272775e39e1d
  Running command git submodule update --init --recursive -q
  From https://github.com/ngoyal2707/Megatron-LM
   * branch            adb23324c222aad0aad89308e70302d996a5eaeb -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 123 kB 6.8 M

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 40.0 MB/s 
     |████████████████████████████████| 895 kB 37.7 MB/s 
     |████████████████████████████████| 3.3 MB 39.1 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip3 install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio===0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html


In [ ]:
import torch
torch.version.cuda


'11.3'

In [ ]:
import torchaudio

In [ ]:
%cd s3prl

[Errno 2] No such file or directory: 's3prl'
/content


In [ ]:
!git clone https://github.com/s3prl/s3prl.git

Cloning into 's3prl'...
remote: Enumerating objects: 13226, done.
remote: Counting objects: 100% (1803/1803), done.
remote: Compressing objects: 100% (372/372), done.
remote: Total 13226 (delta 1611), reused 1483 (delta 1426), pack-reused 11423
Receiving objects: 100% (13226/13226), 121.38 MiB | 24.14 MiB/s, done.
Resolving deltas: 100% (8875/8875), done.


In [ ]:
!pip install -e ./s3prl/

Obtaining file:///content/s3prl
  Cloning https://github.com/huggingface/huggingface_hub.git to /tmp/pip-install-192rst9k/huggingface-hub_f5abc3aad19d469aaba406c57423395f
  Running command git clone -q https://github.com/huggingface/huggingface_hub.git /tmp/pip-install-192rst9k/huggingface-hub_f5abc3aad19d469aaba406c57423395f
     |████████████████████████████████| 38.2 MB 26 kB/s 
     |████████████████████████████████| 120 kB 42.3 MB/s 
     |████████████████████████████████| 11.2 MB 29.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 570 kB 38.2 MB/s 
     |████████████████████████████████| 233 kB 38.3 MB/s 
     |████████████████████████████████| 90 kB 9.3 MB/s 
     |████████████████████████████████| 15.7 MB 10 kB/s 
     |████████████████████████████████| 576 kB 47.3 MB/s 
     |████████████████████████████████| 1.2 MB 30.9 MB/s 
     |█████████████████████

In [ ]:
!pwd

/content


In [ ]:
%cd -q s3prl

In [ ]:
from s3prl.optimizers import get_optimizer
from s3prl.schedulers import get_scheduler

In [ ]:
import s3prl.upstream.distiller.hubconf as distiller
m = distiller.distiller_local('/content/drive/MyDrive/Tesi/states-epoch-859.ckpt')

[UpstreamExpert] - Using the default upstream expert config
[DistillerModel] - Expands the output dimension by 2 times
[DistillerModel] - Pred layers: [4, 12]
[TransformerEncoder] - Attention type = original
[DistillerModel] - Out layer type: expand-last
[DistillerModel] - Inter dim = 768


In [ ]:
from torch import nn
import torch.nn.functional as F

class ClassificationHead(nn.Module):
    """Head for aalbert classification task."""

    def __init__(self):
        super().__init__()
        self.dense = nn.Linear(768, 512)#config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(0.7)
        self.out_proj = nn.Linear(512, 4)#config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Classifier(nn.Module):
    def __init__(self, extractor):
        super().__init__()
        self.extractor = extractor
        self.classifier = ClassificationHead()
    def forward(self, x):
      features = self.extractor(tuple(torch.squeeze(x)))
      features = features['last_hidden_state']
      res = torch.mean(features, dim=1)
      logit = self.classifier(res)
      return logit

In [ ]:
model = Classifier(m)

In [ ]:
from tqdm import tqdm
import math
from torch import autograd
import datetime


def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, lr, last_epoch=0, data_augmentation = False, max_stable=5):
    best_val_loss = 1e9
    counter = 0
    augmented = '_augmented' if data_augmentation else ''
    nan=False
    for epoch in range(last_epoch, epochs):
        correct_predictions = 0
        count=0
        #tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        avg_loss = 0
        model.train()  # train mode
        progress = tqdm(data_tr)
        for batch in progress:
            loss = 0
            # data to device
            X_batch, Y_batch = batch["representation"], batch["target"]
            # print(X_batch.shape)
            X_batch = X_batch.to(DEVICE)
            Y_batch = Y_batch.to(DEVICE)
            # set parameter gradients to zero
            opt.zero_grad()
            # forward
            Y_pred = model(X_batch)
            _, preds = torch.max(Y_pred, dim=1)
            preds = preds.to(DEVICE)
            Y_pred = Y_pred.to(DEVICE)

            loss = loss_fn(Y_pred, Y_batch)# forward-pass
            nan = math.isnan(loss)

            if nan:
              count=count+1
              nan=False
              continue
            correct_predictions += torch.sum(preds == Y_batch)
            progress.set_description(f'Loss: {loss:.2f} #nan->{count}')
            
            loss.backward()  # backward-pass
            opt.step()  # update weights
            if not scheduler is None:
                scheduler.step()
            # calculate loss to show the user
            avg_loss += loss
        avg_loss /= (len(data_tr)-(batch_size*count))
        avg_acc = correct_predictions.double() / ((len(data_tr)-count)*batch_size)
        
        print('loss: %f' % avg_loss)
        print('acc: %f' % avg_acc)
        # show intermediate results


        model.eval()  # testing mode
        val_loss = 0
        val_acc = 0
        print("start validation")
        count_loss_val=0
        progress_val = tqdm(data_val)
        correct_predictions_val = 0
        for v_b in progress_val:
            X_val, Y_val = v_b["representation"], v_b["target"]
            # X_val = X_val.to(DEVICE)#
            # Y_val = Y_val.to(DEVICE)#
            Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
            
            _, preds_val = torch.max(Y_hat, dim=1)
            # preds_val = preds_val.to(DEVICE)#

            loss_val = loss_fn(Y_hat, Y_val)
            nan_loss = math.isnan(loss_val) #check loss==nan
            if nan_loss:
              count_loss_val = count_loss_val+1
              nan_loss = False
              continue
            #se trova un nan, salta tutto il batch
            correct_predictions_val += torch.sum(preds_val == Y_val)
            progress_val.set_description(f'Loss: {loss_val:.2f} #nan->{count_loss_val}')
            val_loss += loss_val

        val_loss /= (len(data_val)-(batch_size*count_loss_val))
        val_acc = correct_predictions_val.double() / ((len(data_val)-count_loss_val)*batch_size)
        print( f"validation loss: {val_loss}")
        print( f"validation accuracy: {val_acc}")
        if val_loss <= best_val_loss and val_loss > 0:
            counter = 0
            now = datetime.datetime.now()
            print(f'Save new model!- {now.day}/{now.month} h{now.hour+2}:{now.minute}')
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'/content/drive/MyDrive/Tesi/DistilHubert/HF_Lung_V1_pretrainedEPOCH859_bandPass_final_80-20_bandPass2_4-class_distilhubert_base_lr={lr}_batch={batch_size}_{freq}mhz_{duration}sec{augmented}.bin')
        else:
            counter += 1
        print("--------------------")
        if counter == max_stable:
            break

----------------------------

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

freq=4
duration=4
train_ds = AudiosDataset(train_4['filename'], train_4['label'])
test_ds = AudiosDataset(test_4['filename'], test_4['label'])

batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test": DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
lr =1e-5
torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["test"], lr)
#16mhz 4secondi tanh dropout=0.5 batch=8 final PRETRAIN 859 EPOCHE 4KHZ

* Epoch 1/100


  0%|          | 0/689 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/content/s3prl/s3prl/upstream/distiller/model.py:274: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  pad_len = (pad_len - k_size) // s_size + 1
Loss: 1.39 #nan->0: 100%|████

loss: 1.181085
acc: 0.502903
start validation


Loss: 0.99 #nan->0: 100%|██████████| 172/172 [00:25<00:00,  6.70it/s]


validation loss: 1.1268384456634521
validation accuracy: 0.5050872093023255
Save new model!- 21/11 h14:26
--------------------
* Epoch 2/100


...

validation loss: 0.9054443836212158
validation accuracy: 0.6707848837209303
--------------------
* Epoch 11/100


Loss: 0.36 #nan->0: 100%|██████████| 689/689 [02:03<00:00,  5.58it/s]


loss: 0.465239
acc: 0.823657
start validation


Loss: 0.79 #nan->0: 100%|██████████| 172/172 [00:26<00:00,  6.56it/s]

validation loss: 0.9422290921211243
validation accuracy: 0.6802325581395349
--------------------
